In [5]:
import copy

# Set search depth - adjust based on performance needs
MAX_DEPTH = 3

class SantoriniBoard:
    def __init__(self):
        # 5x5 grid with [height, worker_owner] in each cell
        # height: 0-4 (0=ground, 4=dome/unplayable)
        # worker_owner: 0=none, 1=player1, 2=player2
        self.grid = [[[0, 0] for _ in range(5)] for _ in range(5)]
        self.last_move = None
        # Each player has 2 workers positioned initially
        self.place_initial_workers()
    
    def place_initial_workers(self):
        # Player 1 workers (could be randomized or predetermined)
        self.grid[1][1][1] = 1
        self.grid[3][3][1] = 1
        # Player 2 workers
        self.grid[1][3][1] = 2
        self.grid[3][1][1] = 2
    
    def copy(self):
        # Create a deep copy of the board
        new_board = SantoriniBoard()
        new_board.grid = copy.deepcopy(self.grid)
        new_board.last_move = self.last_move
        return new_board
    
    def display(self):
        # Display the board in a human-readable format
        print("\n  0 1 2 3 4")
        print(" +---------+")
        for y in range(5):
            print(f"{y}|", end="")
            for x in range(5):
                height = self.grid[y][x][0]
                owner = self.grid[y][x][1]
                
                # Display format: height + owner symbol
                if owner == 0:
                    if height == 4:  # Dome
                        print("D ", end="")
                    else:
                        print(f"{height} ", end="")
                else:
                    worker_symbol = "A" if owner == 1 else "B"
                    print(f"{height}{worker_symbol}", end=" ")
            print("|")
        print(" +---------+")
        print("Legend: [height][worker] (A=Player1, B=Player2, D=Dome)")
    
    def get_possible_moves(self, player):
        moves = []
        # Find all worker positions for this player
        worker_positions = []
        for y in range(5):
            for x in range(5):
                if self.grid[y][x][1] == player:
                    worker_positions.append((x, y))
        
        # For each worker, find all possible move+build combinations
        for wx, wy in worker_positions:
            # Check all adjacent cells for movement
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue  # Skip current position
                    
                    nx, ny = wx + dx, wy + dy
                    # Check if within bounds and no worker and can climb (height difference ≤ 1)
                    if (0 <= nx < 5 and 0 <= ny < 5 and 
                        self.grid[ny][nx][1] == 0 and 
                        self.grid[ny][nx][0] < 4 and
                        self.grid[ny][nx][0] <= self.grid[wy][wx][0] + 1):
                        
                        # Now check all adjacent cells to new position for building
                        for bx in [-1, 0, 1]:
                            for by in [-1, 0, 1]:
                                if bx == 0 and by == 0:
                                    continue  # Skip current position
                                
                                bnx, bny = nx + bx, ny + by
                                # Check if within bounds and no worker and not a complete dome
                                if (0 <= bnx < 5 and 0 <= bny < 5 and 
                                    (bnx != wx or bny != wy) and  # Can't build where we came from
                                    self.grid[bny][bnx][1] == 0 and 
                                    self.grid[bny][bnx][0] < 4):
                                    
                                    moves.append((wx, wy, nx, ny, bnx, bny))
        
        return moves
    
    def apply_move(self, move, player):
        wx, wy, nx, ny, bx, by = move
        # Create a copy of the board
        new_board = self.copy()
        
        # Move worker
        new_board.grid[ny][nx][1] = player
        new_board.grid[wy][wx][1] = 0
        
        # Build at the target location
        new_board.grid[by][bx][0] += 1
        
        # Store the last move
        new_board.last_move = move
        
        return new_board
    
    def is_win(self, move=None):
        # If move is provided, check if that move won
        if move:
            _, _, nx, ny, _, _ = move
            # If a worker moved to level 3, it's a win
            return self.grid[ny][nx][0] == 3
        
        # If no move provided, check if any worker is at level 3
        for y in range(5):
            for x in range(5):
                if self.grid[y][x][1] > 0 and self.grid[y][x][0] == 3:
                    return True
        return False
    
    def is_draw(self):
        # Check if there are no valid moves for either player
        return len(self.get_possible_moves(1)) == 0 and len(self.get_possible_moves(2)) == 0
    
    def evaluate(self, player):
        # Simple evaluation: height advantage and mobility
        score = 0
        opponent = 3 - player
        
        # Calculate height score
        for y in range(5):
            for x in range(5):
                if self.grid[y][x][1] == player:
                    score += self.grid[y][x][0] * 10  # Height is valuable
                elif self.grid[y][x][1] == opponent:
                    score -= self.grid[y][x][0] * 10  # Opponent height is bad
        
        # Calculate mobility score
        player_moves = len(self.get_possible_moves(player))
        opponent_moves = len(self.get_possible_moves(opponent))
        score += (player_moves - opponent_moves) * 2
        
        # Check for win/loss
        if self.is_win():
            for y in range(5):
                for x in range(5):
                    if self.grid[y][x][1] > 0 and self.grid[y][x][0] == 3:
                        if self.grid[y][x][1] == player:
                            return 1000  # Win
                        else:
                            return -1000  # Loss
        
        return score

class SantoriniSearchNode:
    def __init__(self, board, player, depth=0, last_move=None):
        self.board = board
        self.player = player  # Current player's turn
        self.depth = depth
        self.last_move = last_move
        self.children = []
        self.value = None
        
        # Check win condition from previous move
        if last_move and self.board.is_win(last_move):
            self.value = 1000 if depth % 2 == 1 else -1000
            return
            
        # Limit search depth
        if depth >= MAX_DEPTH:
            # Evaluate board position
            self.value = self.board.evaluate(1 if depth % 2 == 0 else 2)
            return
            
        # Generate children
        self.generate_children()
    
    def generate_children(self):
        moves = self.board.get_possible_moves(self.player)
        
        # Handle edge case: no moves available
        if not moves:
            self.value = -1000 if self.depth % 2 == 0 else 1000
            return
            
        for move in moves:
            new_board = self.board.apply_move(move, self.player)
            next_player = 3 - self.player  # Switch players (1->2, 2->1)
            child = SantoriniSearchNode(new_board, next_player, self.depth + 1, move)
            self.children.append(child)
    
    def minimax_value(self, alpha=-float('inf'), beta=float('inf')):
        if self.value is not None:
            return self.value
            
        if not self.children:  # No moves possible
            self.value = -1000 if self.depth % 2 == 0 else 1000
            return self.value
            
        if self.depth % 2 == 0:  # Maximizing player
            best_val = -float('inf')
            for child in self.children:
                val = child.minimax_value(alpha, beta)
                best_val = max(best_val, val)
                alpha = max(alpha, best_val)
                if beta <= alpha:
                    break
            self.value = best_val
        else:  # Minimizing player
            best_val = float('inf')
            for child in self.children:
                val = child.minimax_value(alpha, beta)
                best_val = min(best_val, val)
                beta = min(beta, best_val)
                if beta <= alpha:
                    break
            self.value = best_val
            
        return self.value
    
    def best_move(self):
        self.minimax_value()
        if not self.children:
            return None
            
        if self.depth % 2 == 0:  # Maximizing
            best_children = [c for c in self.children if c.value == max(child.value for child in self.children)]
        else:  # Minimizing
            best_children = [c for c in self.children if c.value == min(child.value for child in self.children)]
            
        # In case of multiple equally good moves, choose one
        import random
        best_child = random.choice(best_children)
        return best_child.last_move

def play_santorini():
    board = SantoriniBoard()
    current_player = 1
    human_player = None
    
    # Ask if the user wants to play as player 1 or 2
    while human_player not in [1, 2]:
        try:
            human_player = int(input("Do you want to play as player 1 or 2? (Enter 1 or 2): "))
            if human_player not in [1, 2]:
                print("Please enter 1 or 2.")
        except ValueError:
            print("Please enter a valid number.")
    
    ai_player = 3 - human_player
    
    print("\nSantorini Game Start!")
    print("How to play:")
    print("- Each turn, you must move one of your workers and then build.")
    print("- To win, move a worker to level 3.")
    print("- You can only move up one level at a time.")
    print("- You can build on any empty space adjacent to your worker's new position.")
    print("- A dome (level 4) cannot be built on or moved to.")
    print("- Your workers are labeled 'A' if you're player 1, 'B' if you're player 2.")
    print("- The number indicates the height of the building.")
    
    while True:
        # Display board
        board.display()
        
        if current_player == human_player:  # Human player's turn
            print(f"\nYour turn (Player {human_player}):")
            
            # Get valid worker selection and move
            valid_move = False
            while not valid_move:
                try:
                    # Select worker
                    print("Select a worker to move (format: x y):")
                    wx, wy = map(int, input().split())
                    
                    # Validate worker selection
                    if not (0 <= wx < 5 and 0 <= wy < 5):
                        print("Invalid coordinates. Please enter values between 0 and 4.")
                        continue
                    if board.grid[wy][wx][1] != human_player:
                        print("That's not your worker. Please select one of your workers.")
                        continue
                    
                    # Select destination
                    print("Select where to move (format: x y):")
                    nx, ny = map(int, input().split())
                    
                    # Validate move
                    if not (0 <= nx < 5 and 0 <= ny < 5):
                        print("Invalid coordinates. Please enter values between 0 and 4.")
                        continue
                    if board.grid[ny][nx][1] != 0:
                        print("That space is occupied. Please select an empty space.")
                        continue
                    if board.grid[ny][nx][0] > 3:
                        print("That space has a dome. You can't move there.")
                        continue
                    if board.grid[ny][nx][0] > board.grid[wy][wx][0] + 1:
                        print("That space is too high to climb. You can only move up one level.")
                        continue
                    if abs(nx - wx) > 1 or abs(ny - wy) > 1:
                        print("You can only move to adjacent spaces.")
                        continue
                    
                    # Select build location
                    print("Select where to build (format: x y):")
                    bx, by = map(int, input().split())
                    
                    # Validate build
                    if not (0 <= bx < 5 and 0 <= by < 5):
                        print("Invalid coordinates. Please enter values between 0 and 4.")
                        continue
                    if board.grid[by][bx][1] != 0:
                        print("That space is occupied. Please select an empty space.")
                        continue
                    if board.grid[by][bx][0] >= 4:
                        print("That space already has a dome. You can't build there.")
                        continue
                    if abs(bx - nx) > 1 or abs(by - ny) > 1:
                        print("You can only build on spaces adjacent to your worker's new position.")
                        continue
                    if bx == wx and by == wy:
                        print("You can't build on the space you moved from.")
                        continue
                    
                    move = (wx, wy, nx, ny, bx, by)
                    valid_move = True
                    
                except ValueError:
                    print("Invalid input. Please enter coordinates as two numbers separated by a space.")
            
            # Apply the move
            board = board.apply_move(move, human_player)
            
        else:  # AI player's turn
            print(f"\nAI's turn (Player {ai_player}):")
            
            # Check if AI has any moves
            ai_moves = board.get_possible_moves(ai_player)
            if not ai_moves:
                print("AI has no valid moves. You win!")
                break
                
            print("AI thinking...")
            search = SantoriniSearchNode(board, ai_player)
            move = search.best_move()
            
            if move:
                wx, wy, nx, ny, bx, by = move
                print(f"AI moves worker from ({wx},{wy}) to ({nx},{ny}) and builds at ({bx},{by})")
                board = board.apply_move(move, ai_player)
            else:
                print("AI has no valid moves. You win!")
                break
            
        # Check win condition
        if board.is_win():
            board.display()
            print(f"\nPlayer {current_player} wins!")
            break
            
        # Check for draw
        if board.is_draw():
            board.display()
            print("\nThe game is a draw! No more moves possible.")
            break
            
        # Switch players
        current_player = 3 - current_player

if __name__ == "__main__":
    play_santorini()


Santorini Game Start!
How to play:
- Each turn, you must move one of your workers and then build.
- To win, move a worker to level 3.
- You can only move up one level at a time.
- You can build on any empty space adjacent to your worker's new position.
- A dome (level 4) cannot be built on or moved to.
- Your workers are labeled 'A' if you're player 1, 'B' if you're player 2.
- The number indicates the height of the building.

  0 1 2 3 4
 +---------+
0|0 0 0 0 0 |
1|0 0A 0 0B 0 |
2|0 0 0 0 0 |
3|0 0B 0 0A 0 |
4|0 0 0 0 0 |
 +---------+
Legend: [height][worker] (A=Player1, B=Player2, D=Dome)

Your turn (Player 1):
Select a worker to move (format: x y):
Select where to move (format: x y):
Select where to build (format: x y):

  0 1 2 3 4
 +---------+
0|0 0 0 0 0 |
1|0 0 0 0B 0 |
2|1A 0 0 0 0 |
3|0 0B 0 0A 0 |
4|0 0 0 0 0 |
 +---------+
Legend: [height][worker] (A=Player1, B=Player2, D=Dome)

AI's turn (Player 2):
AI thinking...
AI moves worker from (1,3) to (1,4) and builds at (0,3)

 

KeyboardInterrupt: Interrupted by user